Meta

In [75]:
import datetime as dt
print(dt.datetime.now())

2026-01-29 15:11:13.226347


# Find a specific dataset
In STAC it is common to search variables and query time/space. However, in some cases you might want to find a specific dataset by its title. 
## Scenario 1: you know in which collection the dataset is stored. 
EDITO STAC has several thematic collections. We have prior knowledge that de dataset we are looking for is stored in <b>emodnet-biology</b> collection.
### Step 1 load the collection
The collection that we are looking for is emodnet-biology.

In [76]:
from pystac_client import Client
from pprint import pprint

URL = "https://api.dive.edito.eu/data/collections"
catalog = Client.open(URL)
collection_emodnet_biology = catalog.get_collection("emodnet-biology")

Inspect the collections

In [77]:
collection_emodnet_biology

<CollectionClient id=emodnet-biology>

## List all items in the collection
As you will see, there are many items in this collection

In [78]:
emodnet_biology_items = list(collection_emodnet_biology.get_items())
for i, item in enumerate(emodnet_biology_items):
    print(f"item {i} out of {len(emodnet_biology_items)}, {item.properties.get('title', '')}")

item 0 out of 1718, Habitat suitability reef-forming species in the North Sea
item 1 out of 1718, Biodiversity data from rocky intertidal zones, surveyed from Scotland to Morocco in 2022 and 2023
item 2 out of 1718, Counts of seabirds, marine mammals and other megafauna during POLARSTERN cruise ANT-XXVIII/5
item 3 out of 1718, Counts of seabirds, marine mammals and other megafauna during POLARSTERN cruise PS83 (ANT-XXIX/10)
item 4 out of 1718, Counts of seabirds, marine mammals and other megafauna during POLARSTERN cruise ANT-XXIX/1
item 5 out of 1718, Counts of seabirds, marine mammals and other megafauna during POLARSTERN cruise ANT-XXVIII/1
item 6 out of 1718, Modeled distribution map of Sea pens and burrowing megafauna in the North East Atlantic in 2021
item 7 out of 1718, Potential catch biomass summed up along the water column for the 30 main commercial fish species from the Atlantic Ocean
item 8 out of 1718, 3D habitat suitability maps of the 30 main commercial fish species from

## Search for the dataset
The dataset we are looking for is this one: https://zenodo.org/records/13589902

In [79]:
search_title = "Koster"

In [80]:
for i, item in enumerate(emodnet_biology_items):
    if search_title in item.properties.get('title'):
        print(item)

<Item id=bdbeb221-7656-52e5-9ade-4b3304db82cd>


In [81]:
results = search_by_title(collection, "Koster")
print("Matches:", len(results))
for item in results:
    print(item.id, "->", item.properties["title"])

Matches: 1
bdbeb221-7656-52e5-9ade-4b3304db82cd -> Koster historical biodiversity assessment


In [82]:
koster_item = results[0]
print(koster_item)

<Item id=bdbeb221-7656-52e5-9ade-4b3304db82cd>


## Inspect the Item
Using help() gives you all info. 

In [83]:
#help(koster_item)

You can also visit the docs: https://pystac.readthedocs.io/en/latest/api/pystac.html#pystac.Item 
<br>
Or inspect object signature

In [84]:
import inspect

import inspect
sig = inspect.signature(type(koster_item))
for name, param in sig.parameters.items():
    print(name, param)

id id: 'str'
geometry geometry: 'dict[str, Any] | None'
bbox bbox: 'list[float] | None'
datetime datetime: 'Datetime | None'
properties properties: 'dict[str, Any]'
start_datetime start_datetime: 'Datetime | None' = None
end_datetime end_datetime: 'Datetime | None' = None
stac_extensions stac_extensions: 'list[str] | None' = None
href href: 'str | None' = None
collection collection: 'str | Collection | None' = None
extra_fields extra_fields: 'dict[str, Any] | None' = None
assets assets: 'dict[str, Asset] | None' = None


## View the properties

In [85]:
koster_item.properties

{'productIdentifier': 'koster_historical_biodiversity_assessment',
 'title': 'Koster historical biodiversity assessment',
 'start_datetime': '1997-08-27T00:00:00.000000Z',
 'end_datetime': '2023-10-09T00:00:00.000000Z',
 'license': 'CC-BY-4.0',
 'provider': 'EMODnet Biology',
 'proj:code': 'EPSG:EPSG:4326',
 'collection': 'emodnet-biology',
 'updated': '2025-03-27T13:37:59.583384Z',
 'created': '2025-03-27T10:44:11.884559Z',
 'likes': 0,
 'comments': 0,
 'owner': '226779592474001643',
 'status': 1,
 'centroid': [0, 0],
 'ssys:targets': ['earth']}

## View the assets

In [86]:
koster_item.assets

{'doi': <Asset href=https://doi.org/10.5281/zenodo.13589902>,
 'wfs': <Asset href=https://geo.vliz.be/geoserver/Dataportal/wfs?service=wfs&version=1.1.0&typeName=eurobis-obisenv_basic&request=GetFeature&outputFormat=text%2Fcsv&viewParams=datasetid%3A8744>,
 'iptdwca': <Asset href=https://ipt.gbif.org.nz/archive.do?r=koster_historical_assessment>,
 'iptresource': <Asset href=https://ipt.gbif.org.nz/resource?r=koster_historical_assessment>,
 'xml': <Asset href=https://emodnet.ec.europa.eu/geonetwork/srv/api/records/6d617269-6e65-696e-666f-000000008744/formatters/xml>,
 'csw': <Asset href=https://emodnet.ec.europa.eu/geonetwork/emodnet/eng/csw?request=GetRecordById&service=CSW&version=2.0.2&elementSetName=full&id=6d617269-6e65-696e-666f-000000008744>}

In [106]:
collections = list(catalog.get_collections())

In [107]:
collections

[<CollectionClient id=animal_tracking_datasets>,
 <CollectionClient id=climate_forecast-age_of_sea_ice>,
 <CollectionClient id=climate_forecast-aggregate_quality_flag>,
 <CollectionClient id=climate_forecast-air_density>,
 <CollectionClient id=climate_forecast-air_pressure>,
 <CollectionClient id=climate_forecast-air_pressure_at_mean_sea_level>,
 <CollectionClient id=climate_forecast-air_temperature>,
 <CollectionClient id=climate_forecast-atmosphere_upward_relative_vorticity>,
 <CollectionClient id=climate_forecast-barotropic_eastward_sea_water_velocity>,
 <CollectionClient id=climate_forecast-barotropic_northward_sea_water_velocity>,
 <CollectionClient id=climate_forecast-beaufort_wind_force>,
 <CollectionClient id=climate_forecast-charnock_coefficient_for_surface_roughness_length_for_momentum_in_air>,
 <CollectionClient id=climate_forecast-concentration_of_colored_dissolved_organic_matter_in_sea_water_expressed_as_equivalent_mass_fraction_of_quinine_sulfate_dihydrate>,
 <CollectionC

In [ ]:
for i, collection in enumerate(collections): 
    print(f"collection {i} out of {len(collections)}")
    for item in collection.get_items():
        print(item)


    


collection 0 out of 445
<Item id=f96c1e37-1a8f-556c-b73f-0d5bdd7c13b2>
<Item id=26ec8eb5-c3a0-5482-bdb1-46ba2e0afd90>
<Item id=501557f0-f75d-54fc-bb76-72302643a1ac>
<Item id=06b96d7a-e75f-52cb-8e9d-d04894dc550f>
<Item id=708d1b3c-61d1-5d9c-82eb-c539ea4b7a9e>
<Item id=f0416a71-d09d-5af5-8068-48fe69484d35>
<Item id=d23d913f-f32b-51e5-b12c-27d70abf0056>
<Item id=a8ed5c47-ec38-5436-a97f-4c889381e21e>
<Item id=d94fc9dd-ed1f-5986-9eb1-ba786bd7f825>
<Item id=380af265-60b6-532f-bf2b-407ae2197c7c>
<Item id=23322141-2976-5509-981b-4ac573c15c00>
<Item id=57cf6b9a-9d74-5751-afea-77c6bafeb329>
<Item id=99a25472-2ab6-5832-aa2e-ff433d38172b>
<Item id=f499a5ad-3dc8-5dc1-8d0c-a025f56c10f7>
<Item id=d0e2491e-a229-5e5a-9d9a-36a69a4278ab>
<Item id=03de998c-84d5-5269-b566-f4ed5029415a>
<Item id=68b8c094-a668-5989-be52-9f4c1b89841f>
<Item id=8d505bfb-101d-5825-abd5-5c30551814bf>
<Item id=07c4a741-c119-509d-9084-dab6de301048>
<Item id=5f017ead-a944-5d02-bcb1-ccf70cb64ec2>
<Item id=e8eec1cf-4625-529c-a4c7-4e5

In [ ]:
emodnet_biology_items = list(collection_emodnet_biology.get_items())
for i, item in enumerate(emodnet_biology_items):
    print(f"item {i} out of {len(emodnet_biology_items)}, {item.properties.get('title', ''